# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-19-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-18-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head(20)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-08-19 04:27:47,33.939110,67.709953,37599,1375,27166,9058.0,Afghanistan,96.585159,3.657012
1,NaN,NaN,NaN,Albania,2020-08-19 04:27:47,41.153300,20.168300,7654,232,3871,3551.0,Albania,265.967058,3.031095
2,NaN,NaN,NaN,Algeria,2020-08-19 04:27:47,28.033900,1.659600,39444,1391,27653,10400.0,Algeria,89.949970,3.526519
3,NaN,NaN,NaN,Andorra,2020-08-19 04:27:47,42.506300,1.521800,1005,53,869,83.0,Andorra,1300.718307,5.273632
4,NaN,NaN,NaN,Angola,2020-08-19 04:27:47,-11.202700,17.873900,1966,90,667,1209.0,Angola,5.981817,4.577823
5,NaN,NaN,NaN,Antigua and Barbuda,2020-08-19 04:27:47,17.060800,-61.796400,93,3,88,2.0,Antigua and Barbuda,94.967731,3.225806
6,NaN,NaN,NaN,Argentina,2020-08-19 04:27:47,-38.416100,-63.616700,305966,6048,223531,76387.0,Argentina,676.979179,1.976690
7,NaN,NaN,NaN,Armenia,2020-08-19 04:27:47,40.069100,45.038200,41846,832,34982,6032.0,Armenia,1412.173321,1.988243
8,NaN,NaN,Australian Capital Territory,Australia,2020-08-19 04:27:47,-35.473500,149.012400,113,3,110,0.0,"Australian Capital Territory, Australia",26.395702,2.654867
9,NaN,NaN,New South Wales,Australia,2020-08-19 04:27:47,-33.868800,151.209300,3966,52,2994,920.0,"New South Wales, Australia",48.854398,1.311145


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37054,37162,37269,37345,37424,37431,37551,37596,37599,37599
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,6411,6536,6676,6817,6971,7117,7260,7380,7499,7654
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,35160,35712,36204,36699,37187,37664,38133,38583,39025,39444


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1312,1328,1344,1354,1363,1363,1370,1375,1375,1375
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,199,200,205,208,213,219,225,228,230,232
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1302,1312,1322,1333,1341,1351,1360,1370,1379,1391


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,25960,26228,26415,26694,26714,26714,27166,27166,27166,27166
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3342,3379,3480,3552,3616,3695,3746,3794,3816,3871
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,24506,24920,25263,25627,26004,26308,26644,27017,27347,27653


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1169,1174,1162,1188,1196,1196,1206,1218,1219,1235
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,3544,3651,3670,3710,3744,3791,3839,3877,3888,3906
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,575,576,579,581,586,590,598,604,605,606


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,21,21,22,22,22,22,22,22,22,22
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,25,25,29,29,29,29,29,29,29,30
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5,5,6,6,6,6,6,6,6,6


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-08-19 04:27:47,32.539527,-86.644082,1235,22,0,1213.0,"Autauga, Alabama, US",2210.528200,1.781377
668,1075.0,Lamar,Alabama,US,2020-08-19 04:27:47,33.779950,-88.096680,247,2,0,245.0,"Lamar, Alabama, US",1789.206809,0.809717
669,1077.0,Lauderdale,Alabama,US,2020-08-19 04:27:47,34.901719,-87.656247,1269,21,0,1248.0,"Lauderdale, Alabama, US",1368.503920,1.654846


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,5482416,171821,1898159
Brazil,3407354,109888,2751246
India,2767253,52888,2037816


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,5482416,171821,1898159,3412436,2020-08-19 04:27:47,37.914208,-91.353830
Brazil,3407354,109888,2751246,546220,2020-08-19 04:27:47,-12.669522,-48.480493
India,2767253,52888,2037816,676549,2020-08-19 04:27:47,23.088275,81.806127
Russia,930276,15836,741045,173395,2020-08-19 04:27:47,54.546312,62.120860
South Africa,592144,12264,485468,94412,2020-08-19 04:27:47,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,638140,11504,0
Florida,579932,9758,0
Texas,568812,10678,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,638140,11504,0,626636,2020-08-19 04:27:47,37.843962,-120.728594
Florida,579932,9758,0,570174,2020-08-19 04:27:47,28.940755,-82.700744
Texas,568812,10678,0,558134,2020-08-19 04:27:47,31.660643,-98.653069
New York,426571,32857,0,393714,2020-08-19 04:27:47,42.671593,-75.579694
Georgia,241677,4794,0,236883,2020-08-19 04:27:47,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,230742,23642,0
California,Los Angeles,224105,5340,0
Florida,Miami-Dade,146990,2126,0
Arizona,Maricopa,129958,2612,0
Illinois,Cook,117171,4974,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,230742,23642,0,207100,2020-08-19 04:27:47,40.767273,-73.971526,36061.0
California,Los Angeles,224105,5340,0,218765,2020-08-19 04:27:47,34.308284,-118.228241,6037.0
Florida,Miami-Dade,146990,2126,0,144864,2020-08-19 04:27:47,25.611236,-80.551706,12086.0
Arizona,Maricopa,129958,2612,0,127346,2020-08-19 04:27:47,33.348359,-112.491815,4013.0
Illinois,Cook,117171,4974,0,112197,2020-08-19 04:27:47,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-16,37596,7380,38583,989,1906,93,294569,41663,23558,23370,...,320343,1440,35329,33755,964,16534,10,1869,9343,5261
2020-08-17,37599,7499,39025,1005,1935,93,299126,41701,23773,23534,...,321064,1457,35702,34802,983,16844,10,1882,9839,5308
2020-08-18,37599,7654,39444,1005,1966,93,305966,41846,23989,23829,...,322177,1485,36352,35697,989,17306,10,1889,9981,5378


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-16,1375,228,1370,53,88,3,5703,818,421,728,...,41451,38,232,281,24,110,1,530,260,132
2020-08-17,1375,230,1379,53,88,3,5814,824,438,729,...,41454,40,236,288,25,113,1,535,264,135
2020-08-18,1375,232,1391,53,90,3,6048,832,450,729,...,41466,40,242,297,26,113,1,537,264,141


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-16,27166,3794,27017,863,628,88,211702,34584,14083,20681,...,1486,1200,30973,22700,456,9838,8,1013,8412,2092
2020-08-17,27166,3816,27347,869,632,88,217850,34655,14536,20765,...,1490,1205,30973,23575,467,9906,8,1045,8575,3848
2020-08-18,27166,3871,27653,869,667,88,223531,34982,14927,20870,...,1493,1219,32062,24561,526,9939,8,1052,8776,4105


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1169,1174,1162,1188,1196,1196,1206,1218,1219,1235
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,222,229,230,230,237,237,239,240,245,247
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1167,1189,1190,1203,1213,1219,1231,1238,1240,1269
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,349,351,354,356,360,364,367,370,370,377
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,2678,2700,2708,2724,2736,2735,2760,2776,2779,2861


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-16,108433,4259,0,193537,52665,623873,53157,50897,16451,49,...,133708,555394,46652,1515,741,106687,67461,8567,65741,3286
2020-08-17,109004,4309,0,194005,53077,629415,53351,51267,16536,49,...,134744,561110,46894,1527,760,107421,67721,8641,66196,3331
2020-08-18,110361,4371,0,194920,53487,638140,53612,51255,16593,49,...,135770,568812,47157,1530,796,108282,68263,8738,66196,3360


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-16        1218    3877     604  469    860     496    792    1919   
2020-08-17        1219    3888     605  474    863     496    795    1936   
2020-08-18        1235    3906     606  476    887     504    796    1982   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-16          854      302  ...     144      6       83       40   
2020-08-17          855      306  ...     146      6       86       40   
2020-08-18          859      313  ...     148      7       91       40   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-16            276   389   276          0       97      7  
2020-08-17            278   390   277          0      100      8  
2020-08-18            279   393   278          0      102      8  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-16,1898,28,0,4506,599,11243,1896,4453,593,0,...,1366,10396,363,58,9,2381,1781,160,1039,30
2020-08-17,1925,28,0,4506,603,11296,1896,4456,593,0,...,1387,10447,364,58,9,2385,1785,160,1039,30
2020-08-18,1936,29,0,4529,619,11504,1899,4456,593,0,...,1424,10678,369,58,9,2396,1808,162,1039,30


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-16          22      29       6    5      5      11     36      19   
2020-08-17          22      29       6    5      5      13     36      21   
2020-08-18          22      30       6    5      5      13     36      23   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-16           38        8  ...       0      0        0        0   
2020-08-17           38        8  ...       0      0        0        0   
2020-08-18           38        8  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-16              0     0     0         29        0      0  
2020-08-17              0     0     0         29        0      0  
2020-08-18              0     0     0         29        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-16,0.001198,0.016529,0.011801,0.000000,0.014369,0.0,0.018917,0.004049,0.011637,0.008240,...,0.003480,0.004184,0.023199,0.035207,0.013670,0.023587,0.0,0.005920,0.017091,0.016422
2020-08-17,0.000080,0.016125,0.011456,0.016178,0.015215,0.0,0.015470,0.000912,0.009126,0.007018,...,0.002251,0.011806,0.010558,0.031018,0.019710,0.018749,0.0,0.006956,0.053088,0.008934
2020-08-18,0.000000,0.020669,0.010737,0.000000,0.016021,0.0,0.022867,0.003477,0.009086,0.012535,...,0.003467,0.019218,0.018206,0.025717,0.006104,0.027428,0.0,0.003719,0.014432,0.013188


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-16,0.00365,0.013333,0.007353,0.0,0.023256,0.0,0.011708,0.001224,0.063131,0.000000,...,0.000121,0.000000,0.031111,0.018116,0.000000,0.018519,0.0,0.003788,0.000000,0.015385
2020-08-17,0.00000,0.008772,0.006569,0.0,0.000000,0.0,0.019463,0.007335,0.040380,0.001374,...,0.000072,0.052632,0.017241,0.024911,0.041667,0.027273,0.0,0.009434,0.015385,0.022727
2020-08-18,0.00000,0.008696,0.008702,0.0,0.022727,0.0,0.040248,0.009709,0.027397,0.000000,...,0.000289,0.000000,0.025424,0.031250,0.040000,0.000000,0.0,0.003738,0.000000,0.044444


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-16,0.0,0.012814,0.013999,0.000000,0.000000,0.060241,0.029193,0.002900,0.033008,0.002618,...,0.000000,0.005025,0.05609,0.043822,0.020134,0.047934,0.0,0.000000,0.043025,0.021983
2020-08-17,0.0,0.005799,0.012215,0.006952,0.006369,0.000000,0.029041,0.002053,0.032166,0.004062,...,0.002692,0.004167,0.00000,0.038546,0.024123,0.006912,0.0,0.031589,0.019377,0.839388
2020-08-18,0.0,0.014413,0.011190,0.000000,0.055380,0.000000,0.026078,0.009436,0.026899,0.005057,...,0.002013,0.011618,0.03516,0.041824,0.126338,0.003331,0.0,0.006699,0.023440,0.066788


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-16,0.007929,0.024783,NaN,0.004583,0.012944,0.009513,0.006399,0.000000,0.003354,0.0,...,0.014885,0.016178,0.007146,0.003976,0.009537,0.008861,0.008612,0.012408,0.010529,0.018283
2020-08-17,0.005266,0.011740,NaN,0.002418,0.007823,0.008883,0.003650,0.007270,0.005167,0.0,...,0.007748,0.010292,0.005187,0.007921,0.025641,0.006880,0.003854,0.008638,0.006921,0.013694
2020-08-18,0.012449,0.014388,NaN,0.004716,0.007725,0.013862,0.004892,-0.000234,0.003447,0.0,...,0.007614,0.013726,0.005608,0.001965,0.047368,0.008015,0.008003,0.011226,0.000000,0.008706


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-16      0.009950  0.009898  0.010033  0.000000  0.007026  0.002020   
2020-08-17      0.000821  0.002837  0.001656  0.010661  0.003488  0.000000   
2020-08-18      0.013126  0.004630  0.001653  0.004219  0.027810  0.016129   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-08-16      0.005076  0.008408  0.000000  0.010033  ...  0.000000   
2020-08-17      0.003788  0.008859  0.001171  0.013245  ...  0.013889   
2020-08-18      0.001258  0.023760  0.004678  0.022876  ...  0.013699   

Province_State                                                              \
Admin2            Platte  Sheridan Sublette Sweetwater     Teton     Uinta   
2020-08-16      0.000000  0.024691      0.0   0.003636  0.010390  0.000000   
2020-08-17      0.000000  0.036145      0.0   0.007246  0.002571  0.003623   
2020-08-18      0.166667  0.058140      0.0   0.003597  0.007692  0.003610   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-08-16            NaN  0.000000  0.000000  
2020-08-17            NaN  0.030928  0.142857  
2020-08-18            NaN  0.020000  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-16,0.001055,0.000000,NaN,0.003117,-0.001667,0.002229,0.000000,0.000000,0.0,NaN,...,0.015613,0.012466,0.000000,0.0,0.0,0.000000,0.008494,0.0000,0.000963,0.0
2020-08-17,0.014226,0.000000,NaN,0.000000,0.006678,0.004714,0.000000,0.000674,0.0,NaN,...,0.015373,0.004906,0.002755,0.0,0.0,0.001680,0.002246,0.0000,0.000000,0.0
2020-08-18,0.005714,0.035714,NaN,0.005104,0.026534,0.018414,0.001582,0.000000,0.0,NaN,...,0.026676,0.022112,0.013736,0.0,0.0,0.004612,0.012885,0.0125,0.000000,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                 \
Admin2         Autauga   Baldwin Barbour Bibb Blount   Bullock Butler   
2020-08-16         0.0  0.000000     0.0  0.0    0.0  0.000000    0.0   
2020-08-17         0.0  0.000000     0.0  0.0    0.0  0.181818    0.0   
2020-08-18         0.0  0.034483     0.0  0.0    0.0  0.000000    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-08-16      0.000000      0.0      0.0  ...     NaN    NaN      NaN   
2020-08-17      0.105263      0.0      0.0  ...     NaN    NaN      NaN   
2020-08-18      0.095238      0.0      0.0  ...     NaN    NaN      NaN   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-16          NaN        NaN   NaN   NaN        0.0      NaN    NaN  
2020-08-17          NaN        NaN   NaN   NaN        0.0      NaN    NaN  
2020-08-18          NaN        NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-16,0.001694,0.018632,0.012328,0.001810,0.016678,0.00136,0.021759,0.004665,0.012115,0.009859,...,0.003568,0.006726,0.021481,0.036813,0.018385,0.022224,1.541976e-18,0.004227,0.020842,0.017682
2020-08-17,0.000887,0.017378,0.011892,0.008994,0.015946,0.00068,0.018614,0.002789,0.010621,0.008438,...,0.002909,0.009266,0.016019,0.033915,0.019047,0.020487,7.709882e-19,0.005591,0.036965,0.013308
2020-08-18,0.000444,0.019024,0.011314,0.004497,0.015984,0.00034,0.020741,0.003133,0.009853,0.010487,...,0.003188,0.014242,0.017113,0.029816,0.012576,0.023957,3.854941e-19,0.004655,0.025699,0.013248


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-16,0.004057,0.019626,0.007148,0.000601,0.022164,4.738712e-38,0.019015,0.002902,0.049221,0.001190,...,0.000163,0.003383,0.026935,0.027154,0.046562,0.015871,0.0,0.002423,0.010558,0.020399
2020-08-17,0.002029,0.014199,0.006859,0.000300,0.011082,2.369356e-38,0.019239,0.005119,0.044800,0.001282,...,0.000118,0.028007,0.022088,0.026033,0.044114,0.021572,0.0,0.005928,0.012971,0.021563
2020-08-18,0.001014,0.011447,0.007780,0.000150,0.016905,1.184678e-38,0.029744,0.007414,0.036099,0.000641,...,0.000204,0.014004,0.023756,0.028641,0.042057,0.010786,0.0,0.004833,0.006486,0.033004


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-16,0.004809,0.015064,0.013511,0.001560,0.020818,0.041644,0.040722,0.006201,0.025250,0.004444,...,0.001565,0.006739,0.051906,0.032939,0.020993,0.034328,9.035018e-21,0.010503,0.043832,0.036273
2020-08-17,0.002404,0.010431,0.012863,0.004256,0.013594,0.020822,0.034881,0.004127,0.028708,0.004253,...,0.002128,0.005453,0.025953,0.035742,0.022558,0.020620,4.517509e-21,0.021046,0.031605,0.437831
2020-08-18,0.001202,0.012422,0.012026,0.002128,0.034487,0.010411,0.030479,0.006781,0.027803,0.004655,...,0.002071,0.008536,0.030556,0.038783,0.074448,0.011976,2.258755e-21,0.013872,0.027523,0.252309


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-16,0.008957,0.023297,NaN,0.004848,0.007656,0.011702,0.006385,0.000468,0.006518,2.981486e-47,...,0.013562,0.014999,0.007913,0.005285,0.028291,0.009279,0.009822,0.015664,0.012060,0.016184
2020-08-17,0.007111,0.017518,NaN,0.003633,0.007739,0.010292,0.005017,0.003869,0.005842,1.490743e-47,...,0.010655,0.012645,0.006550,0.006603,0.026966,0.008079,0.006838,0.012151,0.009490,0.014939
2020-08-18,0.009780,0.015953,NaN,0.004175,0.007732,0.012077,0.004955,0.001817,0.004645,7.453715e-48,...,0.009135,0.013186,0.006079,0.004284,0.037167,0.008047,0.007421,0.011688,0.004745,0.011823


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-16      0.008422  0.011030  0.010037  0.005874  0.008653  0.003214   
2020-08-17      0.004622  0.006934  0.005846  0.008267  0.006071  0.001607   
2020-08-18      0.008874  0.005782  0.003750  0.006243  0.016940  0.008868   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-08-16      0.006482  0.011344  0.001318  0.014917  ...  0.006527   
2020-08-17      0.005135  0.010101  0.001244  0.014081  ...  0.010208   
2020-08-18      0.003196  0.016931  0.002961  0.018478  ...  0.011953   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-08-16      0.012500  0.029719  0.000808   0.008001  0.009644 -0.001304   
2020-08-17      0.006250  0.032932  0.000404   0.007623  0.006107  0.001159   
2020-08-18      0.086458  0.045536  0.000202   0.005610  0.006900  0.002385   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-08-16           -1.0  0.022405  0.098958  
2020-08-17           -1.0  0.026666  0.120908  
2020-08-18           -1.0  0.023333  0.060454  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-16,0.002620,0.010543,NaN,0.008471,0.007820,0.007477,0.001831,0.000133,0.000174,NaN,...,0.014863,0.017710,0.005527,0.000009,0.000061,0.002076,0.008021,0.010430,0.005265,0.002752
2020-08-17,0.008423,0.005271,NaN,0.004235,0.007249,0.006096,0.000916,0.000403,0.000087,NaN,...,0.015118,0.011308,0.004141,0.000004,0.000031,0.001878,0.005133,0.005215,0.002633,0.001376
2020-08-18,0.007068,0.020493,NaN,0.004670,0.016892,0.012255,0.001249,0.000202,0.000043,NaN,...,0.020897,0.016710,0.008939,0.000002,0.000015,0.003245,0.009009,0.008857,0.001316,0.000688


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2020-08-16      0.000756  0.002753  0.003126  0.000585  0.004561   
2020-08-17      0.000378  0.001376  0.001563  0.000292  0.002281   
2020-08-18      0.000189  0.017930  0.000781  0.000146  0.001140   

Province_State                                                            ...  \
Admin2               Bullock    Butler   Calhoun      Chambers  Cherokee  ...   
2020-08-16      1.490122e-09  0.000056  0.004693  1.863288e-08  0.001116  ...   
2020-08-17      9.090909e-02  0.000028  0.054978  9.316438e-09  0.000558  ...   
2020-08-18      4.545455e-02  0.000014  0.075108  4.658219e-09  0.000279  ...   

Province_State Wyoming                                                  \
Admin2            Park Platte Sheridan Sublette Sweetwater Teton Uinta   
2020-08-16         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-17         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-18         NaN    NaN      NaN      NaN        NaN   NaN   NaN   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-08-16       0.002851      NaN    NaN  
2020-08-17       0.001425      NaN    NaN  
2020-08-18       0.000713      NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200818,AK,5177,304832,NaN,47.0,NaN,NaN,NaN,7.0,...,310009,1,0,fc59520a56a12fd11b3c22d44d638c59207c3a94,0,0,0,0,0,NaN
1,20200818,AL,110361,757898,NaN,1280.0,12958.0,NaN,1329.0,NaN,...,868259,11,0,c98faa4cfe97429cc87bf6dca25290c1bc2e1738,0,0,0,0,0,NaN
2,20200818,AR,53487,579575,NaN,492.0,3695.0,NaN,NaN,122.0,...,633062,16,74,1cf18fc9e8912cc5d8e79a3392e75472186fabf8,0,0,0,0,0,NaN
3,20200818,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,80c26eb73b3dd77695b768e4fec36114f19405ea,0,0,0,0,0,NaN
4,20200818,AZ,194920,909077,NaN,1167.0,20878.0,427.0,NaN,260.0,...,1103997,23,122,ae27525e5b4b87412400edf2797fb9cc84722b57,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200818,AK,5177.0,304832.0,NaN,47.0,NaN,NaN,NaN,7.0,...,310009,1,0,fc59520a56a12fd11b3c22d44d638c59207c3a94,0,0,0,0,0,NaN
1,20200818,AL,110361.0,757898.0,NaN,1280.0,12958.0,NaN,1329.0,NaN,...,868259,11,0,c98faa4cfe97429cc87bf6dca25290c1bc2e1738,0,0,0,0,0,NaN
2,20200818,AR,53487.0,579575.0,NaN,492.0,3695.0,NaN,NaN,122.0,...,633062,16,74,1cf18fc9e8912cc5d8e79a3392e75472186fabf8,0,0,0,0,0,NaN
3,20200818,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,80c26eb73b3dd77695b768e4fec36114f19405ea,0,0,0,0,0,NaN
4,20200818,AZ,194920.0,909077.0,NaN,1167.0,20878.0,427.0,NaN,260.0,...,1103997,23,122,ae27525e5b4b87412400edf2797fb9cc84722b57,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-18,AK,5177,304832,NaN,47.0,NaN,NaN,NaN,7.0,NaN,...,310009,1,0,fc59520a56a12fd11b3c22d44d638c59207c3a94,0,0,0,0,0,NaN
2020-08-18,AL,110361,757898,NaN,1280.0,12958.0,NaN,1329.0,NaN,729.0,...,868259,11,0,c98faa4cfe97429cc87bf6dca25290c1bc2e1738,0,0,0,0,0,NaN
2020-08-18,AR,53487,579575,NaN,492.0,3695.0,NaN,NaN,122.0,478.0,...,633062,16,74,1cf18fc9e8912cc5d8e79a3392e75472186fabf8,0,0,0,0,0,NaN
2020-08-18,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,80c26eb73b3dd77695b768e4fec36114f19405ea,0,0,0,0,0,NaN
2020-08-18,AZ,194920,909077,NaN,1167.0,20878.0,427.0,NaN,260.0,NaN,...,1103997,23,122,ae27525e5b4b87412400edf2797fb9cc84722b57,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-18,AK,5177.0,304832.0,NaN,47.0,NaN,NaN,NaN,7.0,NaN,...,310009,1,0,fc59520a56a12fd11b3c22d44d638c59207c3a94,0,0,0,0,0,NaN
2020-08-18,AL,110361.0,757898.0,NaN,1280.0,12958.0,NaN,1329.0,NaN,729.0,...,868259,11,0,c98faa4cfe97429cc87bf6dca25290c1bc2e1738,0,0,0,0,0,NaN
2020-08-18,AR,53487.0,579575.0,NaN,492.0,3695.0,NaN,NaN,122.0,478.0,...,633062,16,74,1cf18fc9e8912cc5d8e79a3392e75472186fabf8,0,0,0,0,0,NaN
2020-08-18,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,80c26eb73b3dd77695b768e4fec36114f19405ea,0,0,0,0,0,NaN
2020-08-18,AZ,194920.0,909077.0,NaN,1167.0,20878.0,427.0,NaN,260.0,NaN,...,1103997,23,122,ae27525e5b4b87412400edf2797fb9cc84722b57,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE